In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# Install dependencies
!pip install openpyxl xgboost -q

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set paths - UPDATE THESE to your Google Drive folder
DRIVE_FOLDER = '/content/drive/MyDrive/satellite_project'  # <-- UPDATE THIS

TRAIN_PATH = f'{DRIVE_FOLDER}/train.xlsx'
TEST_PATH = f'{DRIVE_FOLDER}/test.xlsx'
IMAGE_DIR = f'{DRIVE_FOLDER}/mapbox_images'
OUTPUT_DIR = '/content/outputs'

!mkdir -p {OUTPUT_DIR}

import os
print(f"Train: {TRAIN_PATH} - exists: {os.path.exists(TRAIN_PATH)}")
print(f"Test: {TEST_PATH} - exists: {os.path.exists(TEST_PATH)}")
print(f"Images: {IMAGE_DIR} - exists: {os.path.exists(IMAGE_DIR)}")

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

from xgboost import XGBRegressor

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {DEVICE}")

In [ ]:
# Configuration
CONFIG = {
    'target_col': 'price',
    'lat_col': 'lat',
    'lon_col': 'long',
    'image_size': 224,
    'seed': 42,
    'test_size': 0.2,
    'batch_size': 32,
    'epochs': 15,
    'lr': 5e-4,
    'use_log_target': True,
}

np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])

## 1. Load Data

In [ ]:
train_df = pd.read_excel(TRAIN_PATH, engine='openpyxl')
test_df = pd.read_excel(TEST_PATH, engine='openpyxl')

train_df.columns = [c.strip() for c in train_df.columns]
test_df.columns = [c.strip() for c in test_df.columns]

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

In [ ]:
train_df.head()

## 2. Data Preprocessing

In [ ]:
exclude_cols = {CONFIG['target_col'], 'date', 'id', CONFIG['lat_col'], CONFIG['lon_col']}
feature_cols = [c for c in train_df.columns if c not in exclude_cols and train_df[c].dtype in ['int64', 'float64']]
print(f"Feature columns ({len(feature_cols)}): {feature_cols}")

In [ ]:
train_data, val_data = train_test_split(train_df, test_size=CONFIG['test_size'], random_state=CONFIG['seed'])
train_image_ids = train_data.index.tolist()
val_image_ids = val_data.index.tolist()

print(f"Training: {len(train_data)}, Validation: {len(val_data)}")

In [ ]:
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

X_train = preprocessor.fit_transform(train_data[feature_cols])
X_val = preprocessor.transform(val_data[feature_cols])
X_test = preprocessor.transform(test_df[feature_cols])

if CONFIG['use_log_target']:
    y_train = np.log1p(train_data[CONFIG['target_col']].values)
    y_val = np.log1p(val_data[CONFIG['target_col']].values)
    y_train_original = train_data[CONFIG['target_col']].values
    y_val_original = val_data[CONFIG['target_col']].values
else:
    y_train = train_data[CONFIG['target_col']].values
    y_val = val_data[CONFIG['target_col']].values
    y_train_original = y_train
    y_val_original = y_val

print(f"X_train: {X_train.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")

## 3. XGBoost Model

In [ ]:
xgb_model = XGBRegressor(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=CONFIG['seed'],
    early_stopping_rounds=50,
    eval_metric='rmse',
    tree_method='hist',
    device='cuda' if DEVICE == 'cuda' else 'cpu'
)

xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"\n{model_name}: RMSE=${rmse:,.2f}, R²={r2:.4f}")
    return {'model': model_name, 'RMSE': rmse, 'R2': r2}

xgb_pred_raw = xgb_model.predict(X_val)
if CONFIG['use_log_target']:
    xgb_pred = np.expm1(xgb_pred_raw)
    xgb_results = evaluate_model(y_val_original, xgb_pred, 'XGBoost')
else:
    xgb_results = evaluate_model(y_val, xgb_pred_raw, 'XGBoost')

## 4. Hybrid Model

In [ ]:
class PropertyDataset(Dataset):
    def __init__(self, X_tabular, y=None, image_ids=None, train_mode=True):
        self.X_tab = X_tabular.astype(np.float32)
        self.y = y.astype(np.float32) if y is not None else None
        self.image_ids = image_ids
        
        if train_mode:
            self.transform = T.Compose([
                T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
                T.RandomHorizontalFlip(0.5),
                T.RandomVerticalFlip(0.5),
                T.RandomRotation(15),
                T.ColorJitter(brightness=0.2, contrast=0.2),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.transform = T.Compose([
                T.Resize((CONFIG['image_size'], CONFIG['image_size'])),
                T.ToTensor(),
                T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        
        self.blank = Image.new('RGB', (CONFIG['image_size'], CONFIG['image_size']), (128, 128, 128))
    
    def __len__(self):
        return len(self.X_tab)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx] if self.image_ids is not None else idx
        img_path = os.path.join(IMAGE_DIR, f'img_{int(img_id)}.png')
        
        try:
            img = Image.open(img_path).convert('RGB') if os.path.exists(img_path) else self.blank
        except:
            img = self.blank
        
        img_tensor = self.transform(img)
        tab_tensor = torch.from_numpy(self.X_tab[idx])
        
        if self.y is None:
            return img_tensor, tab_tensor
        return img_tensor, tab_tensor, torch.tensor(self.y[idx])

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, tabular_dim):
        super().__init__()
        
        backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        backbone.fc = nn.Identity()
        self.cnn = backbone
        for param in self.cnn.parameters():
            param.requires_grad = False
        
        self.cnn_processor = nn.Sequential(
            nn.Linear(512, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 64), nn.ReLU()
        )
        
        self.tabular = nn.Sequential(
            nn.Linear(tabular_dim, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 128), nn.BatchNorm1d(128), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(128, 64), nn.ReLU()
        )
        
        self.head = nn.Sequential(
            nn.Linear(128, 64), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, 1)
        )
    
    def forward(self, img, tab):
        with torch.no_grad():
            img_feat = self.cnn(img)
        img_feat = self.cnn_processor(img_feat)
        tab_feat = self.tabular(tab)
        return self.head(torch.cat([img_feat, tab_feat], dim=1)).squeeze(1)

In [ ]:
y_train_hybrid = y_train_original.astype(np.float32)
y_val_hybrid = y_val_original.astype(np.float32)

train_dataset = PropertyDataset(X_train, y_train_hybrid, train_image_ids, True)
val_dataset = PropertyDataset(X_val, y_val_hybrid, val_image_ids, False)

train_loader = DataLoader(train_dataset, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

In [ ]:
model = HybridModel(X_train.shape[1]).to(DEVICE)
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3, weight_decay=1e-4)
criterion = nn.MSELoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

best_rmse = float('inf')
best_state = None

for epoch in range(1, CONFIG['epochs'] + 1):
    model.train()
    for img, tab, y in tqdm(train_loader, desc=f'Epoch {epoch}', leave=False):
        img, tab, y = img.to(DEVICE), tab.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(img, tab), y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
    
    model.eval()
    preds = []
    with torch.no_grad():
        for img, tab, y in val_loader:
            preds.extend(model(img.to(DEVICE), tab.to(DEVICE)).cpu().numpy())
    
    val_rmse = np.sqrt(mean_squared_error(y_val_hybrid, preds))
    scheduler.step(val_rmse)
    
    if val_rmse < best_rmse:
        best_rmse = val_rmse
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        print(f"Epoch {epoch}: Val RMSE=${val_rmse:,.0f} ✓")
    else:
        print(f"Epoch {epoch}: Val RMSE=${val_rmse:,.0f}")

print(f"\nBest Val RMSE: ${best_rmse:,.0f}")

In [ ]:
model.load_state_dict(best_state)
model.eval()
preds = []
with torch.no_grad():
    for img, tab, _ in val_loader:
        preds.extend(model(img.to(DEVICE), tab.to(DEVICE)).cpu().numpy())

hybrid_results = evaluate_model(y_val_original, np.array(preds), 'Hybrid')

## 5. Generate Predictions

In [ ]:
xgb_test_preds_raw = xgb_model.predict(X_test)
test_preds = np.expm1(xgb_test_preds_raw) if CONFIG['use_log_target'] else xgb_test_preds_raw

submission = pd.DataFrame({
    'id': test_df['id'] if 'id' in test_df.columns else range(len(test_preds)),
    'predicted_price': test_preds
})

submission.to_csv(f'{OUTPUT_DIR}/predictions.csv', index=False)
print(f"Saved {len(test_preds)} predictions to {OUTPUT_DIR}/predictions.csv")
submission.head()

In [ ]:
from google.colab import files
files.download(f'{OUTPUT_DIR}/predictions.csv')

## Summary

In [ ]:
results_df = pd.DataFrame([xgb_results, hybrid_results])
print("\nMODEL COMPARISON")
print("="*50)
display(results_df.round(2))

print(f"\nXGBoost RMSE: ${xgb_results['RMSE']:,.0f}")
print(f"Hybrid RMSE: ${hybrid_results['RMSE']:,.0f}")
print("\n✅ Notebook completed!")

In [ ]:
# Install dependencies
!pip install openpyxl xgboost -q

In [ ]:
# Mount Google Drive (Option 1 - Recommended for large files)
from google.colab import drive
drive.mount('/content/drive')

# Set paths - UPDATE THESE to your Google Drive folder
# Example: If your files are in 'My Drive/satellite_project/'
DRIVE_FOLDER = '/content/drive/MyDrive/satellite_project'  # <-- UPDATE THIS

TRAIN_PATH = f'{DRIVE_FOLDER}/train.xlsx'
TEST_PATH = f'{DRIVE_FOLDER}/test.xlsx'
IMAGE_DIR = f'{DRIVE_FOLDER}/mapbox_images'
OUTPUT_DIR = '/content/outputs'

!mkdir -p {OUTPUT_DIR}

In [ ]:
# OR Upload files directly (Option 2 - For smaller files)
# Uncomment below if not using Google Drive

# from google.colab import files
# print("Upload train.xlsx:")
# uploaded = files.upload()
# print("Upload test.xlsx:")
# uploaded = files.upload()

# TRAIN_PATH = 'train.xlsx'
# TEST_PATH = 'test.xlsx'
# IMAGE_DIR = '/content/mapbox_images'  # Upload images separately
# OUTPUT_DIR = '/content/outputs'

In [ ]:
# For images: Upload a zip file and extract
# Uncomment if uploading images as zip

# from google.colab import files
# print("Upload mapbox_images.zip:")
# uploaded = files.upload()
# !unzip -q mapbox_images.zip -d /content/
# IMAGE_DIR = '/content/mapbox_images'

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from xgboost import XGBRegressor

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models

import warnings
warnings.filterwarnings('ignore')

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ Device: {DEVICE}")
print(f"✅ PyTorch: {torch.__version__}")

In [ ]:
# Config
CONFIG = {
    'target_col': 'price',
    'lat_col': 'lat',
    'lon_col': 'long',
    'image_size': 224,
    'seed': 42,
    'test_size': 0.2,
    'batch_size': 32,
    'epochs': 10,
    'lr': 1e-4,
}

np.random.seed(CONFIG['seed'])
torch.manual_seed(CONFIG['seed'])

## 1. Load Data

In [ ]:
# Verify paths exist
print(f"Train file exists: {os.path.exists(TRAIN_PATH)}")
print(f"Test file exists: {os.path.exists(TEST_PATH)}")
print(f"Image dir exists: {os.path.exists(IMAGE_DIR)}")

if os.path.exists(IMAGE_DIR):
    n_images = len([f for f in os.listdir(IMAGE_DIR) if f.endswith('.png')])
    print(f"Number of images: {n_images}")

In [ ]:
# Load data
train_df = pd.read_excel(TRAIN_PATH, engine='openpyxl')
test_df = pd.read_excel(TEST_PATH, engine='openpyxl')

train_df.columns = [c.strip() for c in train_df.columns]
test_df.columns = [c.strip() for c in test_df.columns]

print(f"Train: {train_df.shape}, Test: {test_df.shape}")
train_df.head()

In [ ]:
# Detect correct image naming (by ID or by index)
print("\n🔍 Detecting image naming convention...")
sample_images = sorted(os.listdir(IMAGE_DIR))[:5] if os.path.exists(IMAGE_DIR) else []
print(f"Sample images: {sample_images}")

# Check if images match ID column or row index
if 'id' in train_df.columns:
    test_id = train_df.iloc[0]['id']
    test_by_id = os.path.exists(os.path.join(IMAGE_DIR, f'img_{int(test_id)}.png'))
else:
    test_by_id = False
test_by_idx = os.path.exists(os.path.join(IMAGE_DIR, 'img_0.png'))

USE_ID_FOR_IMAGES = test_by_id

print(f"✅ Images by ID: {test_by_id}, by index: {test_by_idx}")plt.show()

print(f"➡️ Using {'ID column' if USE_ID_FOR_IMAGES else 'row index'} for image lookup")plt.tight_layout()

    ax.axis('off')

def get_image_path_by_id(row):        ax.text(0.5, 0.5, 'No Image', ha='center', va='center')

    """Get image path using ID column."""    else:

    if USE_ID_FOR_IMAGES and 'id' in row.index:        ax.set_title(f'Price: ${row["price"]:,.0f}')

        return os.path.join(IMAGE_DIR, f'img_{int(row["id"])}.png')        ax.imshow(Image.open(img_path))

    return None    if os.path.exists(img_path):

        img_path = os.path.join(IMAGE_DIR, f'img_{i}.png')

# Show sample images    else:

fig, axes = plt.subplots(2, 4, figsize=(16, 8))        img_path = os.path.join(IMAGE_DIR, f'img_{int(row["id"])}.png')

for i, ax in enumerate(axes.flat):    if USE_ID_FOR_IMAGES and 'id' in train_df.columns:
    row = train_df.iloc[i]

## 2. Preprocessing

In [ ]:
# Feature columns
exclude = {CONFIG['target_col'], 'date', 'id', CONFIG['lat_col'], CONFIG['lon_col']}
feature_cols = [c for c in train_df.columns if c not in exclude and train_df[c].dtype in ['int64', 'float64']]
print(f"Features: {feature_cols}")

In [ ]:
# Split
train_data, val_data = train_test_split(train_df, test_size=CONFIG['test_size'], random_state=CONFIG['seed'])

# Store IMAGE IDs (not row indices!) for correct image lookup
if USE_ID_FOR_IMAGES and 'id' in train_df.columns:
    train_image_ids = train_data['id'].tolist()
    val_image_ids = val_data['id'].tolist()
    print("Using 'id' column for image lookup")
else:
    train_image_ids = train_data.index.tolist()
    val_image_ids = val_data.index.tolist()
    print("Using row index for image lookup")

# Preprocess
preprocessor = Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])


X_train = preprocessor.fit_transform(train_data[feature_cols])print(f"X_train: {X_train.shape}")

X_val = preprocessor.transform(val_data[feature_cols])

X_test = preprocessor.transform(test_df[feature_cols])y_val = val_data[CONFIG['target_col']].values

y_train = train_data[CONFIG['target_col']].values

## 3. Model 1: XGBoost (Baseline)

In [ ]:
xgb = XGBRegressor(
    n_estimators=500, max_depth=6, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8, random_state=42,
    early_stopping_rounds=50, eval_metric='rmse',
    tree_method='hist', device='cuda'
)
xgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

In [ ]:
def evaluate(y_true, y_pred, name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n{name}: RMSE=${rmse:,.0f}, MAE=${mae:,.0f}, R²={r2:.4f}")
    return {'model': name, 'RMSE': rmse, 'MAE': mae, 'R2': r2}

xgb_results = evaluate(y_val, xgb.predict(X_val), 'XGBoost (Tabular)')

## 4. Model 2: Hybrid (Tabular + Satellite)

In [ ]:
class PropertyDataset(Dataset):
    def __init__(self, X, y=None, image_ids=None, train_mode=True):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32) if y is not None else None
        self.image_ids = image_ids  # IDs for img_{id}.png lookup
        
        transforms = [T.Resize((224, 224)), T.ToTensor(), T.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])]
        if train_mode:
            transforms.insert(1, T.RandomHorizontalFlip(0.5))
        self.transform = T.Compose(transforms)
        self.blank = Image.new('RGB', (224, 224), (128, 128, 128))
    
    def __len__(self): return len(self.X)
    
    def __getitem__(self, idx):
        # Use image ID (not row index!) for correct image lookup
        img_id = self.image_ids[idx] if self.image_ids is not None else idx
        img_path = os.path.join(IMAGE_DIR, f'img_{int(img_id)}.png')
        img = Image.open(img_path).convert('RGB') if os.path.exists(img_path) else self.blank
        img_t = self.transform(img)

        tab_t = torch.from_numpy(self.X[idx])        return (img_t, tab_t) if self.y is None else (img_t, tab_t, torch.tensor(self.y[idx]))

In [ ]:
class HybridModel(nn.Module):
    def __init__(self, tab_dim):
        super().__init__()
        backbone = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        self.cnn = nn.Sequential(*list(backbone.children())[:-1], nn.Flatten())
        self.tab = nn.Sequential(nn.Linear(tab_dim, 128), nn.ReLU(), nn.Dropout(0.2), nn.Linear(128, 64), nn.ReLU())
        self.head = nn.Sequential(nn.Linear(512+64, 256), nn.ReLU(), nn.Dropout(0.3), nn.Linear(256, 1))
    
    def forward(self, img, tab):
        return self.head(torch.cat([self.cnn(img), self.tab(tab)], dim=1)).squeeze(1)

In [ ]:
# Data loaders with CORRECT image IDs
train_loader = DataLoader(PropertyDataset(X_train, y_train, train_image_ids, True), batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(PropertyDataset(X_val, y_val, val_image_ids, False), batch_size=32, num_workers=2)

# Model
model = HybridModel(X_train.shape[1]).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = nn.MSELoss()

print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Training
best_rmse, best_state = float('inf'), None

for epoch in range(CONFIG['epochs']):
    model.train()
    for img, tab, y in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
        img, tab, y = img.to(DEVICE), tab.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(img, tab), y)
        loss.backward()
        optimizer.step()
    
    # Validate
    model.eval()
    preds = []
    with torch.no_grad():
        for img, tab, _ in val_loader:
            preds.extend(model(img.to(DEVICE), tab.to(DEVICE)).cpu().numpy())
    
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    if rmse < best_rmse:
        best_rmse = rmse
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        print(f"Epoch {epoch+1}: RMSE=${rmse:,.0f} ✓")
    else:
        print(f"Epoch {epoch+1}: RMSE=${rmse:,.0f}")

In [ ]:
# Evaluate best model
model.load_state_dict(best_state)
model.eval()
preds = []
with torch.no_grad():
    for img, tab, _ in val_loader:
        preds.extend(model(img.to(DEVICE), tab.to(DEVICE)).cpu().numpy())

hybrid_results = evaluate(y_val, np.array(preds), 'Hybrid (Tabular + Satellite)')

## 5. Comparison

In [ ]:
results = pd.DataFrame([xgb_results, hybrid_results])
print("\n" + "="*50)
print("MODEL COMPARISON")
print("="*50)
display(results)

improvement = (xgb_results['RMSE'] - hybrid_results['RMSE']) / xgb_results['RMSE'] * 100
print(f"\n📈 Improvement from satellite images: {improvement:.2f}%")

In [ ]:
# Visualization
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].bar(results['model'], results['RMSE'], color=['steelblue', 'coral'])
ax[0].set_title('RMSE (Lower is Better)')
ax[1].bar(results['model'], results['R2'], color=['steelblue', 'coral'])
ax[1].set_title('R² (Higher is Better)')
plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/comparison.png')
plt.show()

## 6. Generate Predictions

In [ ]:
# Test predictions with correct image IDs
if USE_ID_FOR_IMAGES and 'id' in test_df.columns:
    test_image_ids = test_df['id'].tolist()
else:
    test_image_ids = list(range(len(test_df)))
test_loader = DataLoader(PropertyDataset(X_test, None, test_image_ids, False), batch_size=32, num_workers=2)

model.eval()
test_preds = []
with torch.no_grad():
    for img, tab in tqdm(test_loader):
        test_preds.extend(model(img.to(DEVICE), tab.to(DEVICE)).cpu().numpy())

# Save
submission = pd.DataFrame({
    'id': test_df['id'] if 'id' in test_df.columns else range(len(test_preds)),
    'predicted_price': test_preds

})submission.head()

submission.to_csv(f'{OUTPUT_DIR}/predictions.csv', index=False)print(f"✅ Saved to {OUTPUT_DIR}/predictions.csv")

In [ ]:
# Download predictions
from google.colab import files
files.download(f'{OUTPUT_DIR}/predictions.csv')